In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os

In [2]:
import polars as pl
from datetime import datetime, timedelta
import numpy as np

# Importing Data Sources

In [3]:
# Import Datasets
customers_df = pl.scan_csv("/Users/jrdegbe/Desktop/weds/Kaggle-Brazilian-Ecommerce-Prediction-main/data/raw/olist_customers_dataset.csv")
sellers_df = pl.scan_csv("/Users/jrdegbe/Desktop/weds/Kaggle-Brazilian-Ecommerce-Prediction-main/data/raw/olist_sellers_dataset.csv")
ord_items_df = pl.scan_csv("/Users/jrdegbe/Desktop/weds/Kaggle-Brazilian-Ecommerce-Prediction-main/data/raw/olist_order_items_dataset.csv")
products_df = pl.scan_csv("/Users/jrdegbe/Desktop/weds/Kaggle-Brazilian-Ecommerce-Prediction-main/data/raw/olist_products_dataset.csv")
orders_df = pl.scan_csv("/Users/jrdegbe/Desktop/weds/Kaggle-Brazilian-Ecommerce-Prediction-main/data/raw/olist_orders_dataset.csv")
payments_df = pl.scan_csv("/Users/jrdegbe/Desktop/weds/Kaggle-Brazilian-Ecommerce-Prediction-main/data/raw/olist_order_payments_dataset.csv")

# Added Additional Datasets

brazilian_holidays_df = pl.scan_csv("/Users/jrdegbe/Desktop/weds/Kaggle-Brazilian-Ecommerce-Prediction-main/data/raw/Brazilian_Holidays.csv")
brazilian_cities_population_df = pl.scan_csv("/Users/jrdegbe/Desktop/weds/Kaggle-Brazilian-Ecommerce-Prediction-main/data/raw/Population_Brazilian_Cities_V2.csv")

# Feature: Total Payment Value

In [4]:
### Feature: Total Payment Value ###

pay_df = payments_df.lazy().select(pl.col(["order_id","payment_value"])).collect()

print(pay_df)

shape: (103_886, 2)
┌──────────────────────────────────┬───────────────┐
│ order_id                         ┆ payment_value │
│ ---                              ┆ ---           │
│ str                              ┆ f64           │
╞══════════════════════════════════╪═══════════════╡
│ b81ef226f3fe1789b1e8b2acac839d17 ┆ 99.33         │
│ a9810da82917af2d9aefd1278f1dcfa0 ┆ 24.39         │
│ 25e8ea4e93396b6fa0d3dd708e76c1bd ┆ 65.71         │
│ ba78997921bbcdc1373bb41e913ab953 ┆ 107.78        │
│ …                                ┆ …             │
│ 7b905861d7c825891d6347454ea7863f ┆ 96.8          │
│ 32609bbb3dd69b3c066a6860554a77bf ┆ 47.77         │
│ b8b61059626efa996a60be9bb9320e10 ┆ 369.54        │
│ 28bbae6599b09d39ca406b747b6632b1 ┆ 191.58        │
└──────────────────────────────────┴───────────────┘


# Feature: Num items per order


In [5]:
### Feature: Num items per order ### 

num_items_per_order_df = ord_items_df.lazy().select(
    [
        pl.col("order_id"),
        pl.col("order_item_id").unique().count().over("order_id").alias("num_items_per_order")
    
    ]).collect()


print(num_items_per_order_df.head(20))

shape: (20, 2)
┌──────────────────────────────────┬─────────────────────┐
│ order_id                         ┆ num_items_per_order │
│ ---                              ┆ ---                 │
│ str                              ┆ u32                 │
╞══════════════════════════════════╪═════════════════════╡
│ 00010242fe8c5a6d1ba2dd792cb16214 ┆ 1                   │
│ 00018f77f2f0320c557190d7a144bdd3 ┆ 1                   │
│ 000229ec398224ef6ca0657da4fc703e ┆ 1                   │
│ 00024acbcdf0a6daa1e931b038114c75 ┆ 1                   │
│ …                                ┆ …                   │
│ 0009c9a17f916a706d71784483a5d643 ┆ 1                   │
│ 000aed2e25dbad2f9ddb70584c5a2ded ┆ 1                   │
│ 000c3e6612759851cc3cbb4b83257986 ┆ 1                   │
│ 000e562887b1f2006d75e0be9558292e ┆ 1                   │
└──────────────────────────────────┴─────────────────────┘


# Feature: Num categories per order

In [6]:
### Feature: Num categories per order ### 
num_cat_per_order_df = ord_items_df.lazy().join(products_df.lazy(), how="left", on="product_id").select(pl.col(["order_id", "order_item_id", "product_category_name"])).select(
    [
        pl.col("order_id"),
        pl.col("product_category_name").unique().count().over("order_id").alias("num_cat_per_order")
    
    ]).collect()

print(num_cat_per_order_df.head(20))

shape: (20, 2)
┌──────────────────────────────────┬───────────────────┐
│ order_id                         ┆ num_cat_per_order │
│ ---                              ┆ ---               │
│ str                              ┆ u32               │
╞══════════════════════════════════╪═══════════════════╡
│ 00010242fe8c5a6d1ba2dd792cb16214 ┆ 1                 │
│ 00018f77f2f0320c557190d7a144bdd3 ┆ 1                 │
│ 000229ec398224ef6ca0657da4fc703e ┆ 1                 │
│ 00024acbcdf0a6daa1e931b038114c75 ┆ 1                 │
│ …                                ┆ …                 │
│ 0009c9a17f916a706d71784483a5d643 ┆ 1                 │
│ 000aed2e25dbad2f9ddb70584c5a2ded ┆ 1                 │
│ 000c3e6612759851cc3cbb4b83257986 ┆ 1                 │
│ 000e562887b1f2006d75e0be9558292e ┆ 1                 │
└──────────────────────────────────┴───────────────────┘


# Feature: Customer location

In [7]:
### Feature: Customer location ### 

customer_locat_df = orders_df.lazy().select(pl.col(["order_id", "customer_id"])).join(customers_df.lazy().select(pl.col(["customer_id", "customer_city", "customer_state"])), how="left", on="customer_id").collect()

print(customer_locat_df.head(20))

shape: (20, 4)
┌───────────────────────────┬───────────────────────────┬─────────────────────────┬────────────────┐
│ order_id                  ┆ customer_id               ┆ customer_city           ┆ customer_state │
│ ---                       ┆ ---                       ┆ ---                     ┆ ---            │
│ str                       ┆ str                       ┆ str                     ┆ str            │
╞═══════════════════════════╪═══════════════════════════╪═════════════════════════╪════════════════╡
│ e481f51cbdc54678b7cc49136 ┆ 9ef432eb6251297304e76186b ┆ sao paulo               ┆ SP             │
│ f2d6af7                   ┆ 10a928d                   ┆                         ┆                │
│ 53cdb2fc8bc7dce0b6741e215 ┆ b0830fb4747a6c6d20dea0b8c ┆ barreiras               ┆ BA             │
│ 0273451                   ┆ 802d7ef                   ┆                         ┆                │
│ 47770eb9100c2d0c44946d9cf ┆ 41ce2a54c0b03bf3443c3d931 ┆ vianopolis        

# Feature: Number of sellers

In [8]:
### Feature: Number of sellers ### 
num_sellers_per_order_df = ord_items_df.lazy().join(sellers_df.lazy(), how="left", on="seller_id").select([
    pl.col("order_id"),
    pl.col("seller_id").unique().count().over("order_id").alias("num_sellers_per_order")
]
).collect()

print(num_sellers_per_order_df.head(20))

shape: (20, 2)
┌──────────────────────────────────┬───────────────────────┐
│ order_id                         ┆ num_sellers_per_order │
│ ---                              ┆ ---                   │
│ str                              ┆ u32                   │
╞══════════════════════════════════╪═══════════════════════╡
│ 00010242fe8c5a6d1ba2dd792cb16214 ┆ 1                     │
│ 00018f77f2f0320c557190d7a144bdd3 ┆ 1                     │
│ 000229ec398224ef6ca0657da4fc703e ┆ 1                     │
│ 00024acbcdf0a6daa1e931b038114c75 ┆ 1                     │
│ …                                ┆ …                     │
│ 0009c9a17f916a706d71784483a5d643 ┆ 1                     │
│ 000aed2e25dbad2f9ddb70584c5a2ded ┆ 1                     │
│ 000c3e6612759851cc3cbb4b83257986 ┆ 1                     │
│ 000e562887b1f2006d75e0be9558292e ┆ 1                     │
└──────────────────────────────────┴───────────────────────┘


# Feature: Number of cities

In [9]:
### Feature: Number of cities ### 
num_cities_per_order_df = ord_items_df.lazy().join(sellers_df.lazy(), how="left", on="seller_id").select([
    pl.col("order_id"),
    pl.col("seller_city").unique().count().over("order_id").alias("num_seller_cities_per_order")
]
).collect()

print(num_cities_per_order_df.head(20))

shape: (20, 2)
┌──────────────────────────────────┬─────────────────────────────┐
│ order_id                         ┆ num_seller_cities_per_order │
│ ---                              ┆ ---                         │
│ str                              ┆ u32                         │
╞══════════════════════════════════╪═════════════════════════════╡
│ 00010242fe8c5a6d1ba2dd792cb16214 ┆ 1                           │
│ 00018f77f2f0320c557190d7a144bdd3 ┆ 1                           │
│ 000229ec398224ef6ca0657da4fc703e ┆ 1                           │
│ 00024acbcdf0a6daa1e931b038114c75 ┆ 1                           │
│ …                                ┆ …                           │
│ 0009c9a17f916a706d71784483a5d643 ┆ 1                           │
│ 000aed2e25dbad2f9ddb70584c5a2ded ┆ 1                           │
│ 000c3e6612759851cc3cbb4b83257986 ┆ 1                           │
│ 000e562887b1f2006d75e0be9558292e ┆ 1                           │
└──────────────────────────────────┴───────────

# Feature: Order Purchased During User Holiday

In [10]:
# Step 1: Get Customers by Order
get_customer_regions_by_order = orders_df.lazy().select(pl.col(["order_id", "customer_id","order_purchase_timestamp"])).join(customers_df.lazy().select(pl.col(["customer_id", "customer_city", "customer_state"])), how="left", on="customer_id").with_columns(pl.col("order_purchase_timestamp").str.strptime(pl.Date, fmt="%Y-%m-%d %H:%M:%S").cast(pl.Datetime)).collect()

print(get_customer_regions_by_order.head(20))

shape: (20, 5)
┌────────────────────┬────────────────────┬───────────────────┬───────────────────┬────────────────┐
│ order_id           ┆ customer_id        ┆ order_purchase_ti ┆ customer_city     ┆ customer_state │
│ ---                ┆ ---                ┆ mestamp           ┆ ---               ┆ ---            │
│ str                ┆ str                ┆ ---               ┆ str               ┆ str            │
│                    ┆                    ┆ datetime[μs]      ┆                   ┆                │
╞════════════════════╪════════════════════╪═══════════════════╪═══════════════════╪════════════════╡
│ e481f51cbdc54678b7 ┆ 9ef432eb6251297304 ┆ 2017-10-02        ┆ sao paulo         ┆ SP             │
│ cc49136f2d6af7     ┆ e76186b10a928d     ┆ 00:00:00          ┆                   ┆                │
│ 53cdb2fc8bc7dce0b6 ┆ b0830fb4747a6c6d20 ┆ 2018-07-24        ┆ barreiras         ┆ BA             │
│ 741e2150273451     ┆ dea0b8c802d7ef     ┆ 00:00:00          ┆             

/var/folders/j9/57z33pgx66ng5_vmqjhktgz80000gn/T/ipykernel_21299/733967856.py:2: DeprecationWarning: `fmt` is deprecated as an argument to `strptime`; use `format` instead.
  get_customer_regions_by_order = orders_df.lazy().select(pl.col(["order_id", "customer_id","order_purchase_timestamp"])).join(customers_df.lazy().select(pl.col(["customer_id", "customer_city", "customer_state"])), how="left", on="customer_id").with_columns(pl.col("order_purchase_timestamp").str.strptime(pl.Date, fmt="%Y-%m-%d %H:%M:%S").cast(pl.Datetime)).collect()


In [11]:
# Step 2: Get sellers By Order
get_sellers_regions_by_order = orders_df.lazy().select(pl.col(["order_id", "order_purchase_timestamp"])).join(ord_items_df.lazy().select(pl.col(["order_id", "order_item_id", "seller_id"])), how="left", on="order_id").join(sellers_df.lazy().select(pl.col(["seller_id","seller_city","seller_state"])), how="left", on="seller_id").with_columns(pl.col("order_purchase_timestamp").str.strptime(pl.Date, fmt="%Y-%m-%d %H:%M:%S").cast(pl.Datetime)).collect()

print(get_sellers_regions_by_order.head(20))

shape: (20, 6)
┌────────────────┬────────────────┬───────────────┬────────────────┬────────────────┬──────────────┐
│ order_id       ┆ order_purchase ┆ order_item_id ┆ seller_id      ┆ seller_city    ┆ seller_state │
│ ---            ┆ _timestamp     ┆ ---           ┆ ---            ┆ ---            ┆ ---          │
│ str            ┆ ---            ┆ i64           ┆ str            ┆ str            ┆ str          │
│                ┆ datetime[μs]   ┆               ┆                ┆                ┆              │
╞════════════════╪════════════════╪═══════════════╪════════════════╪════════════════╪══════════════╡
│ e481f51cbdc546 ┆ 2017-10-02     ┆ 1             ┆ 3504c0cb71d7fa ┆ maua           ┆ SP           │
│ 78b7cc49136f2d ┆ 00:00:00       ┆               ┆ 48d967e0e4c94d ┆                ┆              │
│ 6af7           ┆                ┆               ┆ 59d9           ┆                ┆              │
│ 53cdb2fc8bc7dc ┆ 2018-07-24     ┆ 1             ┆ 289cdb325fb7e7 ┆ belo ho

/var/folders/j9/57z33pgx66ng5_vmqjhktgz80000gn/T/ipykernel_21299/645534870.py:2: DeprecationWarning: `fmt` is deprecated as an argument to `strptime`; use `format` instead.
  get_sellers_regions_by_order = orders_df.lazy().select(pl.col(["order_id", "order_purchase_timestamp"])).join(ord_items_df.lazy().select(pl.col(["order_id", "order_item_id", "seller_id"])), how="left", on="order_id").join(sellers_df.lazy().select(pl.col(["seller_id","seller_city","seller_state"])), how="left", on="seller_id").with_columns(pl.col("order_purchase_timestamp").str.strptime(pl.Date, fmt="%Y-%m-%d %H:%M:%S").cast(pl.Datetime)).collect()


In [12]:
# Objective: Map against order date (if within the week of the holiday for that city/state/nationwide)
brazilian_holidays_transformed_df = brazilian_holidays_df.with_columns([pl.col("Holiday_Date").str.strptime(pl.Date, fmt="%m/%d/%Y").cast(pl.Datetime)]).with_columns([
    (pl.col("Holiday_Date")-pl.duration(days=3)).alias("Holiday_Week_Start"),
    (pl.col("Holiday_Date")+pl.duration(days=3)).alias("Holiday_Week_End")
])

print(brazilian_holidays_transformed_df)

naive plan: (run LazyFrame.explain(optimized=True) to see the optimized plan)

 WITH_COLUMNS:
 [[(col("Holiday_Date")) - (3.duration([0, 0, 0, 0, 0, 0, 0]).alias("duration"))].alias("Holiday_Week_Start"), [(col("Holiday_Date")) + (3.duration([0, 0, 0, 0, 0, 0, 0]).alias("duration"))].alias("Holiday_Week_End")]
   WITH_COLUMNS:
   [col("Holiday_Date").str.strptime().strict_cast(Datetime(Microseconds, None))]

      CSV SCAN /Users/jrdegbe/Desktop/weds/Kaggle-Brazilian-Ecommerce-Prediction-main/data/raw/Brazilian_Holidays.csv
      PROJECT */6 COLUMNS


/var/folders/j9/57z33pgx66ng5_vmqjhktgz80000gn/T/ipykernel_21299/1566380268.py:2: DeprecationWarning: `fmt` is deprecated as an argument to `strptime`; use `format` instead.
  brazilian_holidays_transformed_df = brazilian_holidays_df.with_columns([pl.col("Holiday_Date").str.strptime(pl.Date, fmt="%m/%d/%Y").cast(pl.Datetime)]).with_columns([


In [13]:
national_holidays_df = brazilian_holidays_transformed_df.filter(pl.col("Type")=="National").select([
    pl.col("Type"),
    pl.col("State"),
    pl.col("City_Lower"),
    pl.col("Holiday_Week_Start"),
    pl.col("Holiday_Week_End")
])

print(national_holidays_df)

naive plan: (run LazyFrame.explain(optimized=True) to see the optimized plan)

 SELECT [col("Type"), col("State"), col("City_Lower"), col("Holiday_Week_Start"), col("Holiday_Week_End")] FROM
  FILTER [(col("Type")) == (Utf8(National))] FROM
   WITH_COLUMNS:
   [[(col("Holiday_Date")) - (3.duration([0, 0, 0, 0, 0, 0, 0]).alias("duration"))].alias("Holiday_Week_Start"), [(col("Holiday_Date")) + (3.duration([0, 0, 0, 0, 0, 0, 0]).alias("duration"))].alias("Holiday_Week_End")]
     WITH_COLUMNS:
     [col("Holiday_Date").str.strptime().strict_cast(Datetime(Microseconds, None))]

        CSV SCAN /Users/jrdegbe/Desktop/weds/Kaggle-Brazilian-Ecommerce-Prediction-main/data/raw/Brazilian_Holidays.csv
        PROJECT */6 COLUMNS


In [14]:
#state_holidays = 
state_holidays_df = brazilian_holidays_transformed_df.filter(pl.col("Type")=="State").select([
    pl.col("Type"),
    pl.col("State"),
    pl.col("City_Lower"),
    pl.col("Holiday_Week_Start"),
    pl.col("Holiday_Week_End")
])

print(state_holidays_df)

naive plan: (run LazyFrame.explain(optimized=True) to see the optimized plan)

 SELECT [col("Type"), col("State"), col("City_Lower"), col("Holiday_Week_Start"), col("Holiday_Week_End")] FROM
  FILTER [(col("Type")) == (Utf8(State))] FROM
   WITH_COLUMNS:
   [[(col("Holiday_Date")) - (3.duration([0, 0, 0, 0, 0, 0, 0]).alias("duration"))].alias("Holiday_Week_Start"), [(col("Holiday_Date")) + (3.duration([0, 0, 0, 0, 0, 0, 0]).alias("duration"))].alias("Holiday_Week_End")]
     WITH_COLUMNS:
     [col("Holiday_Date").str.strptime().strict_cast(Datetime(Microseconds, None))]

        CSV SCAN /Users/jrdegbe/Desktop/weds/Kaggle-Brazilian-Ecommerce-Prediction-main/data/raw/Brazilian_Holidays.csv
        PROJECT */6 COLUMNS


In [15]:
 
city_holidays_df = brazilian_holidays_transformed_df.filter(pl.col("Type")=="City").select([
    pl.col("Type"),
    pl.col("State"),
    pl.col("City_Lower"),
    pl.col("Holiday_Week_Start"),
    pl.col("Holiday_Week_End")
])

print(city_holidays_df)

naive plan: (run LazyFrame.explain(optimized=True) to see the optimized plan)

 SELECT [col("Type"), col("State"), col("City_Lower"), col("Holiday_Week_Start"), col("Holiday_Week_End")] FROM
  FILTER [(col("Type")) == (Utf8(City))] FROM
   WITH_COLUMNS:
   [[(col("Holiday_Date")) - (3.duration([0, 0, 0, 0, 0, 0, 0]).alias("duration"))].alias("Holiday_Week_Start"), [(col("Holiday_Date")) + (3.duration([0, 0, 0, 0, 0, 0, 0]).alias("duration"))].alias("Holiday_Week_End")]
     WITH_COLUMNS:
     [col("Holiday_Date").str.strptime().strict_cast(Datetime(Microseconds, None))]

        CSV SCAN /Users/jrdegbe/Desktop/weds/Kaggle-Brazilian-Ecommerce-Prediction-main/data/raw/Brazilian_Holidays.csv
        PROJECT */6 COLUMNS


In [16]:
# get_customer_regions_by_order
# national_holidays_df
# state_holidays_df
# city_holidays_df

sql = pl.SQLContext()
sql.register("get_customer_regions_by_order",get_customer_regions_by_order.lazy())
sql.register("national_holidays_df",national_holidays_df.lazy())
sql.register("state_holidays_df",state_holidays_df.lazy())
sql.register("city_holidays_df",city_holidays_df.lazy())

<SQLContext [tables:4] at 0x125de1610>

In [17]:
sql.register("get_sellers_regions_by_order",get_sellers_regions_by_order.lazy())

<SQLContext [tables:5] at 0x125de1610>

In [18]:
isNationalHolidayCustomerOrder = sql.query(
    """
    Select 
        order_id,
        order_purchase_timestamp,
        customer_id as user_id,
        customer_city as city,
        customer_state as state,
        Holiday_Week_Start,
        Holiday_Week_End 
    From get_customer_regions_by_order
    cross join national_holidays_df
    where order_purchase_timestamp >= Holiday_Week_Start and Holiday_Week_End >= order_purchase_timestamp
    """
)

isNationalHolidayCustomerOrder

/var/folders/j9/57z33pgx66ng5_vmqjhktgz80000gn/T/ipykernel_21299/453576835.py:1: DeprecationWarning: `SQLContext.query` has been renamed; this redirect is temporary, please use `.execute` instead
  isNationalHolidayCustomerOrder = sql.query(


order_id,order_purchase_timestamp,user_id,city,state,Holiday_Week_Start,Holiday_Week_End
str,datetime[μs],str,str,str,datetime[μs],datetime[μs]


In [19]:
isNationalHolidaySellerOrder = sql.query(
    """
    Select 
        order_id,
        order_purchase_timestamp,
        seller_id as user_id,
        seller_city as city,
        seller_state as state,
        Holiday_Week_Start,
        Holiday_Week_End 
    From get_sellers_regions_by_order
    cross join national_holidays_df
    where order_purchase_timestamp >= Holiday_Week_Start and Holiday_Week_End >= order_purchase_timestamp
    """
)

isNationalHolidaySellerOrder

/var/folders/j9/57z33pgx66ng5_vmqjhktgz80000gn/T/ipykernel_21299/2867264113.py:1: DeprecationWarning: `SQLContext.query` has been renamed; this redirect is temporary, please use `.execute` instead
  isNationalHolidaySellerOrder = sql.query(


order_id,order_purchase_timestamp,user_id,city,state,Holiday_Week_Start,Holiday_Week_End
str,datetime[μs],str,str,str,datetime[μs],datetime[μs]


In [20]:
isStateHolidayCustomerOrder = sql.query(
    """
    Select 
        order_id,
        order_purchase_timestamp,
        customer_id as user_id,
        customer_city as city,
        customer_state as state,
        Holiday_Week_Start,
        Holiday_Week_End 
    From get_customer_regions_by_order
    join state_holidays_df
    on state_holidays_df.State = get_customer_regions_by_order.customer_state
    where order_purchase_timestamp >= Holiday_Week_Start and Holiday_Week_End >= order_purchase_timestamp
    """
)

isStateHolidayCustomerOrder

/var/folders/j9/57z33pgx66ng5_vmqjhktgz80000gn/T/ipykernel_21299/831457002.py:1: DeprecationWarning: `SQLContext.query` has been renamed; this redirect is temporary, please use `.execute` instead
  isStateHolidayCustomerOrder = sql.query(


order_id,order_purchase_timestamp,user_id,city,state,Holiday_Week_Start,Holiday_Week_End
str,datetime[μs],str,str,str,datetime[μs],datetime[μs]


In [21]:
isStateHolidaySellerOrder = sql.query(
    """
    Select 
        order_id,
        order_purchase_timestamp,
        seller_id as user_id,
        seller_city as city,
        seller_state as state,
        Holiday_Week_Start,
        Holiday_Week_End
    From get_sellers_regions_by_order
    join state_holidays_df
    on state_holidays_df.State = get_sellers_regions_by_order.seller_state
    where order_purchase_timestamp >= Holiday_Week_Start and Holiday_Week_End >= order_purchase_timestamp
    """
)

isStateHolidaySellerOrder

/var/folders/j9/57z33pgx66ng5_vmqjhktgz80000gn/T/ipykernel_21299/2132418320.py:1: DeprecationWarning: `SQLContext.query` has been renamed; this redirect is temporary, please use `.execute` instead
  isStateHolidaySellerOrder = sql.query(


order_id,order_purchase_timestamp,user_id,city,state,Holiday_Week_Start,Holiday_Week_End
str,datetime[μs],str,str,str,datetime[μs],datetime[μs]


In [22]:
isCityHolidayCustomerOrder = sql.query(
    """
    Select 
        order_id,
        order_purchase_timestamp,
        customer_id as user_id,
        customer_city as city,
        customer_state as state,
        Holiday_Week_Start,
        Holiday_Week_End 
    From get_customer_regions_by_order
    join city_holidays_df
    on city_holidays_df.City_Lower = get_customer_regions_by_order.customer_city
    where order_purchase_timestamp >= Holiday_Week_Start and Holiday_Week_End >= order_purchase_timestamp
    """
)

isCityHolidayCustomerOrder

/var/folders/j9/57z33pgx66ng5_vmqjhktgz80000gn/T/ipykernel_21299/3354409191.py:1: DeprecationWarning: `SQLContext.query` has been renamed; this redirect is temporary, please use `.execute` instead
  isCityHolidayCustomerOrder = sql.query(


order_id,order_purchase_timestamp,user_id,city,state,Holiday_Week_Start,Holiday_Week_End
str,datetime[μs],str,str,str,datetime[μs],datetime[μs]


In [23]:
isCityHolidaySellerOrder = sql.query(
    """
    Select 
        order_id,
        order_purchase_timestamp,
        seller_id as user_id,
        seller_city as city,
        seller_state as state,
        Holiday_Week_Start,
        Holiday_Week_End
    From get_sellers_regions_by_order
    join city_holidays_df
    on city_holidays_df.City_Lower = get_sellers_regions_by_order.seller_city
    where order_purchase_timestamp >= Holiday_Week_Start and Holiday_Week_End >= order_purchase_timestamp
    """
)

isCityHolidaySellerOrder

/var/folders/j9/57z33pgx66ng5_vmqjhktgz80000gn/T/ipykernel_21299/4054698386.py:1: DeprecationWarning: `SQLContext.query` has been renamed; this redirect is temporary, please use `.execute` instead
  isCityHolidaySellerOrder = sql.query(


order_id,order_purchase_timestamp,user_id,city,state,Holiday_Week_Start,Holiday_Week_End
str,datetime[μs],str,str,str,datetime[μs],datetime[μs]


In [24]:
allHolidaysByOrderAll = pl.concat([isNationalHolidaySellerOrder,isStateHolidaySellerOrder,isCityHolidaySellerOrder,isNationalHolidayCustomerOrder,isStateHolidayCustomerOrder,isCityHolidayCustomerOrder])

allHolidaysByOrderAll

order_id,order_purchase_timestamp,user_id,city,state,Holiday_Week_Start,Holiday_Week_End
str,datetime[μs],str,str,str,datetime[μs],datetime[μs]


# Feature: Order purchased during seller history